In [8]:
import os
import pandas as pd
import shutil
from sklearn.model_selection import train_test_split

In [9]:
df_train = pd.read_csv("../data/train.csv")
df_train.head()

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target
0,ISIC_2637011,IP_7279968,male,45.0,head/neck,unknown,benign,0
1,ISIC_0015719,IP_3075186,female,45.0,upper extremity,unknown,benign,0
2,ISIC_0052212,IP_2842074,female,50.0,lower extremity,nevus,benign,0
3,ISIC_0068279,IP_6890425,female,45.0,head/neck,unknown,benign,0
4,ISIC_0074268,IP_8723313,female,55.0,upper extremity,unknown,benign,0


In [10]:
os.listdir("../data/interim/maligno/")[0:5]

['ISIC_0149568.jpg',
 'ISIC_0188432.jpg',
 'ISIC_0207268.jpg',
 'ISIC_0232101.jpg',
 'ISIC_0247330.jpg']

In [11]:
def copiar_imagens(diretorio_origem, diretorio_treino, diretorio_validacao, ids, df_train):
    # Criação dos diretórios de destino, se ainda não existirem
    os.makedirs(diretorio_treino, exist_ok=True)
    os.makedirs(diretorio_validacao, exist_ok=True)
    
    # Dividir a lista de IDs dos pacientes em conjuntos de treinamento e validação
    train_ids, val_ids = train_test_split(ids, test_size=0.25, random_state=42)

    # Iterar sobre as imagens e verificar se o paciente correspondente está no conjunto de treinamento ou validação
    for image_file in os.listdir(diretorio_origem):
        image_name = os.path.splitext(image_file)[0]
        patient_id = df_train[df_train['image_name'] == image_name]['patient_id'].values[0]

        # Verificar se o paciente está no conjunto de treinamento ou validação
        if patient_id in train_ids:
            destino = diretorio_treino
        elif patient_id in val_ids:
            destino = diretorio_validacao
        else:
            continue  # Pular a imagem caso o paciente não esteja no conjunto de treinamento nem validação

        # Caminho completo da imagem de origem
        caminho_origem = os.path.join(diretorio_origem, image_file)

        # Caminho completo da imagem de destino
        caminho_destino = os.path.join(destino, image_file)

        # Copiar a imagem para o diretório de destino correspondente
        shutil.copyfile(caminho_origem, caminho_destino)

# Caminho para o diretório de origem das imagens malignas
diretorio_origem_maligno = "../data/interim/maligno/"
diretorio_treino_maligno = "../data/interim/treino/maligno/"
diretorio_validacao_maligno = "../data/interim/validacao/maligno/"

maligno_ids = df_train[df_train['target'] == 1]['patient_id'].unique()

# Copiar as imagens malignas para os diretórios de treinamento e validação
copiar_imagens(diretorio_origem_maligno, diretorio_treino_maligno, diretorio_validacao_maligno, maligno_ids, df_train)

# Caminho para o diretório de origem das imagens benignas
diretorio_origem_benigno = "../data/interim/benigno/"
diretorio_treino_benigno = "../data/interim/treino/benigno/"
diretorio_validacao_benigno = "../data/interim/validacao/benigno/"

benignas_ids = df_train[df_train['target'] == 0]['patient_id'].unique()

# Copiar as imagens benignas para os diretórios de treinamento e validação
copiar_imagens(diretorio_origem_benigno, diretorio_treino_benigno, diretorio_validacao_benigno, benignas_ids, df_train)
